<a href="https://colab.research.google.com/github/tejas5t6/project-using-cnn/blob/main/cats_vs_dog_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ── STEP 1: Install required libraries
!pip install tensorflow tensorflow-datasets gradio --quiet

# ── STEP 2: Import modules
import tensorflow as tf
import tensorflow_datasets as tfds
import gradio as gr
import os
import matplotlib.pyplot as plt

# ── STEP 3: Mount Google Drive to save/load model
from google.colab import drive
drive.mount('/content/drive')

model_path = "/content/drive/MyDrive/cat_dog_mobilenetv2_model.h5"

# ── STEP 4: Preprocessing function
def preprocess_binary(image, label):
    is_dog = tf.cast(label >= 12, tf.int64)  # Oxford IIIT Pet: dog labels ≥ 12
    image = tf.image.resize(image, (180, 180))
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    return image, is_dog

# ── STEP 5: Load and preprocess dataset
(train_ds, val_ds), info = tfds.load(
    'oxford_iiit_pet',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)

train_ds = train_ds.map(preprocess_binary).shuffle(1000).batch(32).prefetch(1)
val_ds = val_ds.map(preprocess_binary).batch(32).prefetch(1)

# ── STEP 6: Build or load model
if not os.path.exists(model_path):
    print("🔧 Training model from scratch using MobileNetV2...")

    base_model = tf.keras.applications.MobileNetV2(
        input_shape=(180, 180, 3),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False  # Freeze base

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Train the model and store history
    history = model.fit(train_ds, validation_data=val_ds, epochs=5)

    # Plot accuracy and loss
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.tight_layout()
    plt.show()

    model.save(model_path)
    print("✅ Model saved to Drive.")
else:
    print("📁 Loading model from Drive...")
    model = tf.keras.models.load_model(model_path)
    print("✅ Model loaded.")

# ── STEP 7: Define prediction function
def predict_image(img):
    img = tf.image.resize(img, (180, 180))
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    img = tf.expand_dims(img, 0)
    pred = float(model.predict(img)[0][0])
    if abs(pred - 0.5) < 0.2:
        return "❌ Invalid image (not a cat or dog)"
    label = "Dog 🐶" if pred > 0.5 else "Cat 🐱"
    confidence = pred if pred > 0.5 else 1 - pred
    return f"✅ {label} ({confidence * 100:.1f}% confidence)"

# ── STEP 8: Launch Gradio app
gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="numpy", label="Take or Upload Image", sources=["upload", "webcam"]),
    outputs=gr.Textbox(label="Prediction"),
    title="Cat vs Dog Classifier 🐾",
    description="Webcam enabled | Model saved to Google Drive"
).launch()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📁 Loading model from Drive...


✅ Model loaded.
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://32a9fdc83f6b446e82.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
